<a href="https://colab.research.google.com/github/Tomo-for-lab/Code-of-automating-DE/blob/main/data%20extraction/data_extraction_modified.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparation

In [ ]:
!pip install -q openpyxl anthropic weave python-docx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 877.6/877.6 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.2/29.2 MB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.8/327.8 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 42.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.9/202.9 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.0/74.0 kB 381.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.1/65.1 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

In [ ]:
import re
import ipykernel
from notebook.notebookapp import list_running_servers
import requests
from urllib.parse import urljoin
import json
import os
from google.colab import auth
from google.colab import userdata
from googleapiclient.discovery import build
import pandas as pd
import openpyxl
import base64
import csv
import numpy as np
import copy
import glob
import time
from docx import Document

from openai import OpenAI
from openai import AzureOpenAI
from tqdm import tqdm

Change the working directory to the path of the current notebook

In [ ]:
from google.colab import drive
# Mount Google Drive to access files stored in it
drive.mount('/content/drive')


def get_notebook_path():
    # Get the path of the current Jupyter notebook
    kernel_id = re.search('kernel-(.*).json', ipykernel.get_connection_file()).group(1)
    servers = list_running_servers()
    for ss in servers:
        response = requests.get(urljoin(ss['url'], 'api/sessions'), params={'token': ss.get('token', '')})
        for nn in json.loads(response.text):
            if nn['kernel']['id'] == kernel_id:
                relative_path = nn['notebook']['path']
                return os.path.join(ss['notebook_dir'], relative_path)

def get_folder_path(folder_id):
    # Recursively get the full path of a folder given its ID
    if folder_id:
        folder = drive_service.files().get(fileId=folder_id, fields="name, parents").execute()
        folder_name = folder.get('name')
        parents = folder.get('parents')
        if parents:
            parent_path = get_folder_path(parents[0])
            return parent_path + '/' + folder_name
        else:
            return folder_name
    return ''

def get_file_path(file_id):
    # Recursively get the full path of a file given its ID
    file = drive_service.files().get(fileId=file_id, fields="name, parents").execute()
    file_name = file.get('name')
    parents = file.get('parents')
    if parents:
        parent_id = parents[0]
        parent_path = get_folder_path(parent_id)
        return parent_path
    else:
        return file_name

# Get the path of the current notebook
notebook_path = get_notebook_path()


# Authenticate and initialize the Google Drive API
auth.authenticate_user()
drive_service = build('drive', 'v3')

# Extract the file ID from the notebook path (assumes file ID is part of the path)
file_id = re.search(r'fileId=(\w+)', notebook_path).group(1)

# Get the full path of the file using its ID
file_path = get_file_path(file_id)
if 'マイドライブ' in file_path:
    converted_path = re.sub(r'(^|/)マイドライブ($|/)', '/content/drive/MyDrive/', file_path)
elif 'MyDrive' in file_path:
    converted_path = re.sub(r'(^|/)MyDrive($|/)', '/content/drive/MyDrive/', file_path)
else:
    converted_path = '/content/drive/MyDrive/' + file_path

# Change the working directory to the converted path
os.chdir(converted_path)

Mounted at /content/drive


In [ ]:
import weave

# Preparation of Azure for GPT

In [ ]:
#deploy names
deployment_ids = ["gpt-4o","gpt-4-turbo","gpt-3.5-turbo","text-embedding-ada-002"]

#If you use Azure
#key
API_KEY = userdata.get('Azure_API_key')
RESOURCE_ENDPOINT = userdata.get('RESOURCE_ENDPOINT')
AZURE_API_version = userdata.get('AZURE_API_version')

#client
client = AzureOpenAI(
    # https://learn.microsoft.com/en-us/azure/ai-services/openai/reference#rest-api-versioning
    api_version=AZURE_API_version,
    # https://learn.microsoft.com/en-us/azure/cognitive-services/openai/how-to/create-resource?pivots=web-portal#create-a-resource
    azure_endpoint=RESOURCE_ENDPOINT,
    api_key=API_KEY
)


# Define required function

## Function to convert text data in JSON and table data into string variable.

In [ ]:
def extract_sentence(pdf_name):
  # Construct the path to the JSON file associated with the given PDF
  json_name=pdf_name+"/structuredData.json"

  a = open(json_name)# Open and load the JSON file
  j = json.load(a)
  method_flag=1
  extracted_sentence=""# Initialize a string to store the extracted sentences
  previous_item_class=0# To track the class of the previous item and determine if a newline is needed

  # Iterate through each element in the JSON file
  for item in j["elements"] :
    # Check if the item is a heading
    if ("//Document/H" in item['Path']) or (re.search(r"//Document/Sect.*/H", item['Path'])) or (re.search(r"//Document/Aside.*/H", item['Path'])):
      if previous_item_class!=1:
        extracted_sentence+="\n"
        previous_item_class=1
      if "Text" in item:
        if ("reference" == item["Text"].strip().lower()) or ("references" == item["Text"].strip().lower()) or ("appendix" == item["Text"].strip().lower()):
          extracted_sentence+="\n"
          break;
        extracted_sentence+=item["Text"]
        extracted_sentence+="\n"

    # Check if the item is a table and convert table data to text
    elif (re.search("//Document/Table.*" ,item['Path'])) or (re.search(r"//Document/Sect.*/Table", item['Path'])) or (re.search(r"//Document/Aside.*/Table", item['Path'])):#表データを文章に置き換える
      if previous_item_class!=2:
        extracted_sentence+="\n"
        previous_item_class=2
      if "filePaths" in item:
        for t in item["filePaths"]:
          if ".xlsx" in t:
            extracted_sentence+=t.replace(".xlsx", "").split("/")[-1]
            extracted_sentence+="\n"
            t=pdf_name+"/"+t
            # Load the Excel file
            df_t = pd.read_excel(t)
            # Remove carriage returns from column names and data
            df_t.columns = [col.replace('_x000D_', '') for col in df_t.columns]
            df_t = df_t.replace('_x000D_', '', regex=True)
            # Get column names as a string
            column_names = ' '.join(df_t.columns)

            extracted_sentence+=df_t.to_csv(index=False)
            extracted_sentence+="\n"

    # Check if the item is a footnote
    elif (re.search("//Document/Footnote\.*",item['Path'])) or (re.search(r"//Document/Sect.*/Footnote", item['Path'])) or (re.search(r"//Document/Aside.*/Footnote", item['Path'])):
      if previous_item_class!=5:
        extracted_sentence+="\n"
        previous_item_class=5
      if "Text" in item:
        extracted_sentence+=item["Text"]
        extracted_sentence+="\n"

    # Check if the item is a paragraph
    elif ("//Document/P" in item['Path']) or (re.search(r"//Document/Sect.*/P", item['Path'])) or (re.search(r"//Document/Aside.*/P", item['Path'])):
      if previous_item_class!=3:
        extracted_sentence+="\n"
        previous_item_class=3
      if not("/Figure" in item["Path"]):
        if "Text" in item:
          extracted_sentence+=item["Text"]

    # Check if the item is a list
    elif ("//Document/L" in item['Path']) or (re.search(r"//Document/Sect.*/L", item['Path'])) or (re.search(r"//Document/Aside.*/L", item['Path'])):
      if previous_item_class!=4:
        extracted_sentence+="\n"
        previous_item_class=4
      if "/Lbl" in item["Path"]:
        extracted_sentence+="\n"
      if "Text" in item:
        extracted_sentence+=item["Text"]


  # Close the JSON file
  a.close()
  return extracted_sentence

## Function to convert text data in JSON into string variable.

In [ ]:
def extract_sentence_text(pdf_name):
  # Construct the path to the JSON file associated with the given PDF
  json_name=pdf_name+"/structuredData.json"

  a = open(json_name)# Open and load the JSON file
  j = json.load(a)
  method_flag=1
  extracted_sentence="" # Initialize a string to store the extracted sentences
  previous_item_class=0 # To track the class of the previous item and determine if a newline is needed

  # Iterate through each element in the JSON file
  for item in j["elements"] :
    # Check if the item is a heading
    if ("//Document/H" in item['Path']) or (re.search(r"//Document/Sect.*/H", item['Path'])) or (re.search(r"//Document/Aside.*/H", item['Path'])):
      if previous_item_class!=1:
        extracted_sentence+="\n"
        previous_item_class=1
      if "Text" in item:
        # Check for specific keywords to stop extracting sentences
        if ("reference" == item["Text"].strip().lower()) or ("references" == item["Text"].strip().lower()) or ("appendix" == item["Text"].strip().lower()):
          extracted_sentence+="\n"
          break;
        extracted_sentence+=item["Text"]
        extracted_sentence+="\n"

    # Check if the item is a paragraph
    elif ("//Document/P" in item['Path']) or (re.search(r"//Document/Sect.*/P", item['Path'])) or (re.search(r"//Document/Aside.*/P", item['Path'])):
      if previous_item_class!=3:
        extracted_sentence+="\n"
        previous_item_class=3
      if not("/Figure" in item["Path"]):
        if "Text" in item:
          extracted_sentence+=item["Text"]

    # Check if the item is a list
    elif ("//Document/L" in item['Path']) or (re.search(r"//Document/Sect.*/L", item['Path'])) or (re.search(r"//Document/Aside.*/L", item['Path'])):
      if previous_item_class!=4:
        extracted_sentence+="\n"
        previous_item_class=4
      if "/Lbl" in item["Path"]:
        extracted_sentence+="\n"
      if "Text" in item:
        extracted_sentence+=item["Text"]

    # Check if the item is a footnote
    elif (re.search("//Document/Footnote\.*",item['Path'])) or (re.search(r"//Document/Sect.*/Footnote", item['Path'])) or (re.search(r"//Document/Aside.*/Footnote", item['Path'])):
      if previous_item_class != 5:
        extracted_sentence+="\n"
        previous_item_class=5
      if "Text" in item:
        extracted_sentence+=item["Text"]
        extracted_sentence+="\n"

  # Close the JSON file
  a.close()
  return extracted_sentence

## Function to encode images into data URL for input to GPT

In [ ]:
import base64
from mimetypes import guess_type

# Function to encode a local image into data URL
def local_image_to_data_url(image_path):
    # Guess the MIME type of the image based on the file extension
    mime_type, _ = guess_type(image_path)
    if mime_type is None:
        mime_type = 'application/octet-stream'  # Default MIME type if none is found

    # Read and encode the image file
    with open(image_path, "rb") as image_file:
        base64_encoded_data = base64.b64encode(image_file.read()).decode('utf-8')

    # Construct the data URL
    return f"data:{mime_type};base64,{base64_encoded_data}"

## Function for GPT function calling

In [ ]:
@weave.op() # 🐝
def GPT_function_calling(tools, messages, tools_name):
  response = client.chat.completions.create(
        model=deployment_ids[0],
        messages=messages,
        tools=tools,
        tool_choice =  {"type": "function", "function": {"name": tools_name}},
        temperature=0
    )
  return response

# To download data

In [ ]:
# Initialize an empty list to store the list of RCTs for evaluation data.
eval_trials = []

with open('eval_trials_list.txt', 'r') as file:
    for line in file:
        eval_trials.append(json.loads(line.strip()))

# Initialize an empty list to store the list of RCTs for training data.
sample_trials = []

with open('sample_trials_list.txt', 'r') as file:
    for line in file:
        sample_trials.append(json.loads(line.strip()))

#Data extraction

## What LLMs should be aware of when extracting data (free text)

In [ ]:
attention_query = """
#####Attention######

# From which paper to extract data
The primary paper is named AuthorYYYY, the registry ULR AuthorYYYY_NCT1234, the protocol AuthorYYYY_pr, the appendix/supplement files AuthorYYY_app or AuthorYYY_supp, and the secondary papers AuthorYYYY_s1,2,…
You can mainly extract data from primary paper.

#Mixed model repeated measures
- When to be careful
"mixed model", "mixed model repeated measures (MMRM)", "linear mixed model", "growth curve analysis ", "hierarchical model", etc.
If SEs are reported instead of SDs, be especially careful, as MMRM is highly likely.
- What to look out for in evaluation and data extraction
MMRM is said to be a method to estimate the effect of missing values of repeatedly measured outcomes without bias by making the assumption of missing at random
So, N (e.g., Severity_ep_n) in the extraction on outcomes should be the N that was analyzed (=in principle, number randomized; number analyzed for statistical analysis excluding early dropouts). Note that this is different from the number of people for whom outcome measures were available. (In the Christensen2016 example, 1149 patients were randomly assigned and 581 were available at 6 weeks for primary outcome measurement. (N should also be number randomized when calculating SD using SD=SE*sqrt(N).


# How to extract data for each arm
First, extract the data for each arm.
If the arm is further divided into several groups, merge these groups and calculate the integrated data.
If you cannot extract data by arm, extract data for all participants.
If you cannot find any data to extract or you do not have to extract data, output "-1".
For variables related to CBT component, you have to output 1 or 0.
# Which measure data you will take
You have to extract data (mean, SD, number, etc.) in the measure you previously extracted.

# The measure abbreviations
ISI: Insomnia Severity Index
SCI: Sleep Condition Indicator
FOSQ: Functional Outcomes of Sleep Questionnaire
ESS: Epworth Sleepiness Scale
PSQI: Pittsburgh Sleep Quality Index
AIS: Athens Insomnia Scale
WASO: Wake After Sleep Onset
SOL: Sleep Onset Latency

# How to calculate SD
You have to calculate SD and output as number (do not output as formula).
SD=SE*sqrt(n)
　If mixed model repeated measures were used, n is the number of people randomized or included in the analysis.
SD=(upper limit of [100-x]%CI - mean)*sqrt(n)/ NORM.S.INV([100-x/2]/100)
  If 90%CI, then ((upper limit of 90%CI - mean)*sqrt(n)/ norm.s.inv(0.95))
If 95%CI, then ((upper limit of 95%CI - mean)*sqrt(n)/ norm.s.inv(0.975))
If 99%CI, then ((upper limit of 99%CI - mean)*sqrt(n)/ norm.s.inv(0.995))
SD= (Q3 − Q1)/1.35
"""

#Re-check and Re-extract prompting

## Function to create tools for data extraction

In [ ]:
def make_tools_DE(optimized_all_fields):
  properties_dict = {item['name']: {k: v for k, v in item.items() if k != 'name'} for item in optimized_all_fields}
  name_list = [item['name'] for item in optimized_all_fields]

  tools = [{'type': 'function',
      'function':
          {'name': 'extract_study_features',
          'description':"""Extracts key features from a user-inputted research article for systematic review purposes.
          The primary paper is named AuthorYYYY, the registry ULR AuthorYYYY_NCT1234, the protocol AuthorYYYY_pr, the appendix/supplement files AuthorYYY_app or AuthorYYY_supp, and the secondary papers AuthorYYYY_s1,2,…
          You can mainly extract data from primary paper.""",
          'parameters': {
              'type': 'object',
            'properties': {}
          }
      }
  }
  ]


  tools[0]['function']['parameters']['required'] = name_list
  tools[0]['function']['parameters']['properties'] = {key: properties_dict[key] for key in name_list if key in properties_dict}

  return tools

## Function to create tools for re-checking

In [ ]:
def make_tools_recheck(all_fields, Non_DE_name_list):
  original_properties_dict = {item['name']: {k: v for k, v in item.items() if k != 'name'} for item in all_fields}
  check_optimized_all_fields = copy.deepcopy(all_fields)
  for item in check_optimized_all_fields:
    # Output with True or False
    item['type'] = 'boolean'
  check_properties_dict = {item['name']: {k: v for k, v in item.items() if k != 'name'} for item in check_optimized_all_fields}

  description_tools = [{'type': 'function',
      'function':
          {'name': 'determine_data_presence',
           'description': """You have to determine whether you can extract the data corresponding to each variable from the inputted paper of a randomized controlled trial. True if the data to be extracted can be found, False if not.""",
          'parameters': {
              'type': 'object',
            'properties': {}
          }
      }
  }
  ]

  description_tools[0]['function']['parameters']['required'] = Non_DE_name_list
  description_tools[0]['function']['parameters']['properties'] = {key: check_properties_dict[key] for key in Non_DE_name_list if key in check_properties_dict}
  return description_tools

## Function to create tools for re-extracting

In [ ]:
def make_tools_DE_reextract(optimized_all_fields,name_list, start, end):
  properties_dict = {item['name']: {k: v for k, v in item.items() if k != 'name'} for item in optimized_all_fields}

  tools = [{'type': 'function',
      'function':
          {'name': 'extract_study_features',
          'description':"""Extracts key features from a user-inputted research article for systematic review purposes.
          The primary paper is named AuthorYYYY, the registry ULR AuthorYYYY_NCT1234, the protocol AuthorYYYY_pr, the appendix/supplement files AuthorYYY_app or AuthorYYY_supp, and the secondary papers AuthorYYYY_s1,2,…
          You can mainly extract data from primary paper.""",
          'parameters': {
              'type': 'object',
            'properties': {}
          }
      }
  }
  ]


  splitted_name_list = copy.deepcopy(name_list[start:end])
  tools[0]['function']['parameters']['required'] = splitted_name_list
  tools[0]['function']['parameters']['properties'] = {key: properties_dict[key] for key in splitted_name_list if key in properties_dict}

  return tools

In [ ]:
weave.init("data_extraction_after_RSM")

Logged in as Weights & Biases user: taka-too-6t.
View Weave data at https://wandb.ai/srws-psg/data_extraction_after_RSM/weave


In [ ]:
@weave.op() # 🐝
def GPT_chat(tools, messages, tools_name):
  response = client.chat.completions.create(
        model=deployment_ids[0],
        messages=messages,
        tools=tools,
        tool_choice =  {"type": "function", "function": {"name": tools_name}},
        temperature=0
    )
  return response

In [ ]:
#Specify developed description obtained by which method is used
shot_name = "chat"
#Specify the path where the results are to be stored
log_dir_name = "extracted_data_log_GPT_v8"

# Iterate over the number of evaluation data and folds
for n_index in range(5,6):
  for fold_index in range(10):
    for eval_index in range(5):
      eval_name=eval_trials[n_index][fold_index][eval_index]
      print("n_index: ", n_index)
      print("fold_index: ", fold_index)
      print("eval_index: ", eval_index)
      extracted_data = {}#For storing extracted data
      output_extracted_data = {} #For storing output data
############################################
      # Define tools for extracting arm information
      tools = [{'type': 'function',
          'function':
              {'name': 'extract_arm',
              'description': 'Extract the number of the arms and each arm name',
              'parameters': {
                  'type': 'object',
                'properties': {
                    "arms_num":
                    {"type": "number",
                    "description": "Answer the number of the arms"
                    },
                    "arm_name_1":
                    {"type": "string",
                    "description": "Answer arm name. If exceed the number of the arms, answer 'nan'."
                    },

                    "arm_name_2":
                    {"type": "string",
                    "description": "Answer arm name. If exceed the number of the arms, answer 'nan'."
                    },

                    "arm_name_3":
                    {"type": "string",
                    "description": "Answer arm name. If exceed the number of the arms, answer 'nan'."
                    },

                    "arm_name_4":
                    {"type": "string",
                    "description": "Answer arm name. If exceed the number of the arms, answer 'nan'."
                    },

                    "arm_name_5":
                    {"type": "string",
                    "description": "Answer arm name. If exceed the number of the arms, answer 'nan'."
                    }
                },
              "required": [
                  "arms_num",
                  "arm_name_1",
                  "arm_name_2",
                  "arm_name_3",
                  "arm_name_4",
                  "arm_name_5"
                  ]
              }
          }
      }
      ]
#######################################################
      # Create messages for extracting arm information
      messages_arm=[]
      system_content = """You are a systematic reviewer.You have to extract arms(i.e., interventions and controls) from the inputted paper of a randomized controlled trial.\n
      Use the extract_arm tool in your response.      """
      user_content = """You have to extract arms(i.e., interventions and controls) from the inputted paper of a randomized controlled trial.\n
      Use the extract_arm tool in your response.
      """
      png_files_sum = 0

      # Create input text from the paper
      root_dir = f"IncludedTrials/{eval_name}" # Root directory containing the trial files
      file_to_find = "structuredData.json" # JSON file to find in directories

      user_content += f"paper_name: {eval_name} \n"
      for dirpath, dirnames, filenames in os.walk(root_dir):# Process each document in the directory
        if (dirpath == root_dir) or (dirpath == root_dir+f"/{eval_name}"):# Only main documents
          if file_to_find in filenames:
            main_paper_txt= extract_sentence_text(dirpath)
            user_content += main_paper_txt

      # Check if main_paper_txt is empty
      if main_paper_txt == "":
        # Generate the log file name, including date and time
        log_file_name = f"{log_dir_name}/{shot_name}/{n_index}_paper/{fold_index}_fold/error_log_{eval_index}_no_paper_text.txt"
        # Define the error message
        error_message = f"An error occurred: no_paper_text"
        print(error_message)
        # Write the error message to the log file
        with open(log_file_name, 'w') as file:
            file.write(error_message)
        # Skip to the next evaluation RCT
        continue;

      messages_arm.append({"role": "system", "content": system_content})
      messages_arm.append({"role": "user", "content": user_content})

############################################
      # Get response from GPT model
      try:
        tools_name = "extract_arm"
        response_arm = GPT_function_calling(tools, messages_arm, tools_name)
      except Exception as e:
        log_file_name = f"{log_dir_name}/{shot_name}/{n_index}_paper/{fold_index}_fold/error_log_{eval_index}_response_arm.txt"
        error_message = f"An error occurred: {str(e)}"
        print(error_message)
        with open(log_file_name, 'w') as file:
            file.write(error_message)
        continue;

############################################
      # Parse the response
      try:
        # Load the JSON response from the GPT output
        output_dict_arm = json.loads(response_arm.choices[0].message.tool_calls[0].function.arguments)#文章で出力されるため、改行で区切る
        # Extract the number of arms from the JSON dictionary
        arms_num = output_dict_arm["arms_num"]

        arms_list = []
        # Loop through each arm and append its name to the arms_list
        for arm_num in range(arms_num):
          arms_list.append(output_dict_arm[f"arm_name_{arm_num+1}"])
        print("arms_num: ", arms_num)
        print("arms_list: ", arms_list)

      except Exception as e:
        log_file_name = f"{log_dir_name}/{shot_name}/{n_index}_paper/{fold_index}_fold/error_log_{eval_index}_output_dict_arm.txt"
        error_message = f"An error occurred: {str(e)}"
        print(error_message)
        with open(log_file_name, 'w') as file:
            file.write(error_message)
        continue;

############################################
      # Process each arm
      for arm_num in range(arms_num):
        print("arm_num: ", arm_num)
############################################
        # Create a dictionary to store the output
        # Define the path to the JSON file
        json_file_path = 'original_description_insomnia.json'

        # Load the JSON data from the file
        with open(json_file_path, 'r') as file:
            data = json.load(file)
        all_fields = [field for field in data]

        # Initialize the output dictionary with field names as keys and "None" as values
        output = {item['name']: "None" for item in all_fields}

############################################
        # Create tools from optimized description
        try:
          # Determine the log path based on n_index (the number of RCTs for training data)
          if n_index == 0:
            log_path = f'optimized_description_log_v3/{shot_name}/{n_index}_paper/{fold_index}_fold/optimized_all_fields_{n_index}*.json'
          else:
            log_path = f'optimized_description_log_v3/{shot_name}/{n_index}_paper/{fold_index}_fold/optimized_all_fields_{n_index-1}*.json'
          file_paths = glob.glob(log_path)

          for file_path in file_paths:
              with open(file_path, 'r') as file:
                  optimized_all_fields = json.load(file)

          name_list = [item['name'] for item in optimized_all_fields]

          #Add options
          for item in optimized_all_fields:
            if item['name'] == 'ind_clu':
              if not('[individual, cluster]' in item['description']):
                item['description'] += ' You have to choose from the options [individual, cluster].'
            if item['name'] == 'Single_multi':
              if not('[single, multi]' in item['description']):
                item['description'] += ' You have to choose from the options [single, multi].'
            if item['name'] == 'Insomnia_diagnosis':
              if not('[1. formal_DSM, 1. formal_ICSD, 1. formal_ICD, 1. formal_Edinger2004, 2. informal_ISI, 2. informal_SCI, 2. informal_PSQI, 2. informal_AIS, 2. informal_other_self_reported_scales, 3. others]' in item['description']):
                item['description'] += ' You have to choose from the options [1. formal_DSM, 1. formal_ICSD, 1. formal_ICD, 1. formal_Edinger2004, 2. informal_ISI, 2. informal_SCI, 2. informal_PSQI, 2. informal_AIS, 2. informal_other_self_reported_scales, 3. others].'
            if item['name'] == 'Comorbidities':
              if not('[0. primary_insomnia, 1. primary_secondary_mixed, 2. psy_MDD, 2. psy_anxiety, 2. psy_PTSD, 2. psy_schizophrenia_or_psychosis, 2. psy_bipolar, 2. psy_substance_dependence, 2. psy_others, 2. psy_mixed, 3. phy_dialysis, 3. phy_heart_failure, 3. phy_diabetes, 3. phy_fibromyalgia, 3. phy_cancer, 3. phy_steroid, 3. phy_pregnancy_perinatal, 3. phy_menopause, 3. phy_others, 3. phy_mixed, 4. psy_phy_mixed]' in item['description']):
                item['description'] += ' You have to choose from the options [0. primary_insomnia, 1. primary_secondary_mixed, 2. psy_MDD, 2. psy_anxiety, 2. psy_PTSD, 2. psy_schizophrenia_or_psychosis, 2. psy_bipolar, 2. psy_substance_dependence, 2. psy_others, 2. psy_mixed, 3. phy_dialysis, 3. phy_heart_failure, 3. phy_diabetes, 3. phy_fibromyalgia, 3. phy_cancer, 3. phy_steroid, 3. phy_pregnancy_perinatal, 3. phy_menopause, 3. phy_others, 3. phy_mixed, 4. psy_phy_mixed].'
            if item['name'] == 'scale_used':
              if not('[ISI, SCI, FOSQ, ESS, PSQI, AIS, other_self_reported_scales, WASO_SOL, WASO, SOL]' in item['description']):
                item['description'] += ' You have to choose from the options [ISI, SCI, FOSQ, ESS, PSQI, AIS, other_self_reported_scales, WASO_SOL, WASO, SOL].'
            if item['name'] == 'Severity_scale':
              if not('[ISI, SCI, FOSQ, ESS, PSQI, AIS, other_self_reported_scales, WASO_SOL, WASO, SOL]' in item['description']):
                item['description'] += ' You have to choose from the options [ISI, SCI, FOSQ, ESS, PSQI, AIS, other_self_reported_scales, WASO_SOL, WASO, SOL].'
            if item['name'] == 'r_long_scale_used':
              if not('[ISI, SCI, FOSQ, ESS, PSQI, AIS, other_self_reported_scales, WASO_SOL, WASO, SOL]' in item['description']):
                item['description'] += ' You have to choose from the options [ISI, SCI, FOSQ, ESS, PSQI, AIS, other_self_reported_scales, WASO_SOL, WASO, SOL].'

        except Exception as e:
          extracted_data[arms_list[arm_num]] = copy.deepcopy(output)
          log_file_name = f"{log_dir_name}/{shot_name}/{n_index}_paper/{fold_index}_fold/error_log_{eval_index}_data_extraction_armnum_{arm_num}.txt"
          error_message = f"An error occurred: {str(e)}"
          print(error_message)
          with open(log_file_name, 'w') as file:
              file.write(error_message)
          continue;

        ############################################
        # Input all variables at once and extract data (all-in-one data extraction)
        # Create messages for data extraction
        description_tools = make_tools_DE(optimized_all_fields)

        messages_DE = []
        system_content_DE = """You are a systematic reviewer and extract data from RCT.
        """
        query_DE = f"""You have to extract features in the {arms_list[arm_num]} arm from the inputted paper of a randomized controlled trial.
        Use the extract_study_features tool in your response.\n
        """
        query_DE += attention_query
        query_DE += f"trial_name: {eval_name} \n \n "
        query_DE += "The extract_study_features:"
        query_DE += json.dumps(description_tools)

        last_query = attention_query

        messages_GPT = []

        content_prompt_GPT = [] # List to hold content prompts for GPT

        png_files_sum = 0 # Sum of PNG files processed

        # Create input text from the paper
        root_dir = f"IncludedTrials/{eval_name}" # Root directory containing the trial files
        file_to_find = "structuredData.json" # JSON file to find in directories

        content_prompt_GPT.append({"type": "text", "text": query_DE})

        # Extract sentences from each document in the directory
        for dirpath, dirnames, filenames in os.walk(root_dir):
          if eval_name in dirnames:# Process the main paper first
            dirnames.remove(eval_name)
            dirnames.insert(0, eval_name)
          if file_to_find in filenames:
            paper_query = dirpath.split("/")[-1]
            paper_query += extract_sentence(dirpath)
            content_prompt_GPT.append({"type": "text", "text":paper_query})

            # Filter and add PNG files to the content prompt
            if os.path.exists(f"{dirpath}/figures"):
              png_files = [file for file in os.listdir(f"{dirpath}/figures") if file.endswith('.png')]

              for i in range(len(png_files)):
                if png_files_sum >= 10:# Limit the number of images to 10
                  break;
                content_prompt_GPT.append({"type": "image_url", "image_url": {"url": local_image_to_data_url(f"{dirpath}/figures/"+png_files[i])}})
                png_files_sum += 1

        # Process txt and docx files in the root directory
        txt_docx_query = "" # String to hold txt and docx file contents
        txt_docx_flag = False  #Flag to check if txt or docx files are found

        for file_name in os.listdir(root_dir):
            if file_name.endswith('.txt'):
                txt_docx_flag = True
                txt_docx_query += "txt_name: "
                txt_docx_query += "txt_name: " + file_name + "\n"
                file_path = os.path.join(root_dir, file_name)
                with open(file_path, 'r', encoding='utf-8') as file:
                    txt_docx_query += file.read()
            elif file_name.endswith('.docx'):
                txt_docx_flag = True
                txt_docx_query += "docx_name: "
                txt_docx_query += "docx_name: " + file_name + "\n"
                file_path = os.path.join(root_dir, file_name)
                doc = Document(file_path)
                full_text = []
                for para in doc.paragraphs:
                    full_text.append(para.text)
                for table in doc.tables:
                    for row in table.rows:
                        for cell in row.cells:
                            full_text.append(cell.text)
                txt_docx_query += '\n'.join(full_text)

        if txt_docx_flag:
          content_prompt_GPT.append({"type": "text", "text":txt_docx_query})


        content_prompt_GPT.append({"type": "text", "text": last_query})

        messages_GPT.append({"role": "system", "content": system_content_DE})
        messages_GPT.append({"role": "user", "content": content_prompt_GPT})

############################################
        # Get response from GPT model
        tools_name = "extract_study_features"
        response_GPT = GPT_chat(description_tools, messages_GPT, tools_name)
        output_dict = json.loads(response_GPT.choices[0].message.tool_calls[0].function.arguments)

        # Store the results
        DE_name_list = []
        Non_DE_name_list = []
        print(output_dict)
        for key in output_dict.keys():
          if str(output_dict[key]) != "-1":
            DE_name_list.append(key)
          else:
            Non_DE_name_list.append(key)
            output[key] = -1
        print("DE_name_list: ", DE_name_list)
        print("Non_DE_name_list: ", Non_DE_name_list)

############################################
#For variables in the Non_DE_name_list, input the batch and determine whether the data exists again (Re-check)
        # Check for the presence of data for variables in Non_DE_name_list
        if Non_DE_name_list:
          # Create description tools for rechecking
          description_tools = make_tools_recheck(optimized_all_fields, Non_DE_name_list)

          # Initialize the messages for rechecking
          messages_recheck = []
          system_content_recheck = """You are a systematic reviewer. You try extracting data from papers.
          """

          query_GPT = f"""You have to determine whether you can extract the data corresponding to each variable in the {arms_list[arm_num]} from the inputted paper of a randomized controlled trial. True if the data to be extracted can be found, False if not.　If you do not know, assume False."""
          query_GPT += attention_query
          query_GPT += f"trial_name: {eval_name} \n \n "

          query_GPT += "The determine_data_presence: \n"
          query_GPT += json.dumps(description_tools)

          # Modify the content prompt for rechecking
          content_prompt_recheck = copy.deepcopy(content_prompt_GPT)
          content_prompt_recheck.pop(0)
          content_prompt_recheck.insert(0, {"type": "text", "text": query_GPT})

          messages_recheck.append({"role": "system", "content": system_content_recheck})
          messages_recheck.append({"role": "user", "content": content_prompt_recheck})

          tools_name = "determine_data_presence"

          # Call GPT function for rechecking
          response_recheck = GPT_chat(description_tools, messages_recheck, tools_name)

          # Parse the JSON response from rechecking
          output_recheck_dict = json.loads(response_recheck.choices[0].message.tool_calls[0].function.arguments)

          # Reset Non_DE_name_list
          Non_DE_name_list = []

          # Store the obtained results
          for key in output_recheck_dict.keys():
              if key in output.keys():
                if not(output_recheck_dict[key]): # If data is not present
                  Non_DE_name_list.append(key)
                  output[key] = -1 # Mark as -1 indicating data not found

          #Include those for which data is determined to exist into DE_name_list
          DE_name_list.extend([key for key in output_recheck_dict.keys() if output_recheck_dict[key]])

          # Create a dictionary to store the index of each name in name_list
          name_index = {name: index for index, name in enumerate(name_list)}

          # Sort the extracted_name_list based on their index in name_list
          sorted_DE_list = sorted(DE_name_list, key=lambda name: name_index[name])
          DE_name_list = sorted_DE_list

          print("DE_name_list: ", DE_name_list)
          print("Non_DE_name_list: ", Non_DE_name_list)
    ############################################
        #Extract data for every 4 variables in DE_name_list (Re-extract)

        # Initialize messages and a list to track differences for DE extraction
        messages_DE = []
        different_DE_dict = []
        system_content_DE = """You are a systematic reviewer.
        Use the extract_study_features tool in your response.
        """
        query_DE = f"""You have to extract features in the {arms_list[arm_num]} arm from the inputted paper of a randomized controlled trial.
        Use the extract_study_features tool in your response.\n
        """
        query_DE += attention_query
        query_DE += f"trial_name: {eval_name} \n \n "

        # Create and modify the content prompt for DE extraction
        content_prompt_DE = copy.deepcopy(content_prompt_GPT)
        content_prompt_DE.pop(0)
        content_prompt_DE.insert(0, {"type": "text", "text": query_DE})

        # Append initial system, user and assistant messages to messages_DE
        messages_DE.append({"role": "system", "content": system_content_DE})
        messages_DE.append({"role": "user", "content": content_prompt_DE})
        messages_DE.append({"role": "assistant", "content": "Sure. What variable do you want to extract data from?"})

        # Loop through DE_name_list in chunks of 4 and perform data extraction
        for split in range(((len(DE_name_list))//4+1)):
          start = split*4
          end = min(len(DE_name_list), (split+1)*4)

          # Create description tools for the current chunk of DE_name_list
          description_tools = make_tools_DE_reextract(optimized_all_fields,DE_name_list, start, end)
          tools_name = "extract_study_features"

          messages_DE.append({"role": "user", "content": f"Please extract the data for {DE_name_list}"})

          # Call GPT function for DE extraction and append the response to messages_DE
          response_DE = GPT_chat(description_tools, messages_DE, tools_name)
          messages_DE.append({"role":"assistant", "content": response_DE.choices[0].message.tool_calls[0].function.arguments})

          # Parse the JSON response from DE extraction
          output_DE_dict = json.loads(response_DE.choices[0].message.tool_calls[0].function.arguments)

          # Store the obtained results and track differences
          for key in output_DE_dict.keys():
              if key in output.keys():
                  output[key] = output_DE_dict[key]
              if output_DE_dict[key] != output_dict[key]:
                different_DE_dict.append(key)

        # Store the updated output in extracted_data with the current arm name
        extracted_data[arms_list[arm_num]] = copy.deepcopy(output)
        print(different_DE_dict)

      ############################################
      # Write the extraction results of the current evaluation paper to a JSON file
      file_name = f"{log_dir_name}/{shot_name}/{n_index}_paper/{fold_index}_fold/extracted_data_{eval_index}.json"
      output_extracted_data[eval_name] = copy.deepcopy(extracted_data)
      print("log has been created: ", extracted_data)

      # Write the output_extracted_data to the specified JSON file
      with open(file_name, 'w') as file:
          json.dump(output_extracted_data, file, indent=4)


n_index:  5
fold_index:  1
eval_index:  0
🍩 https://wandb.ai/srws-psg/data_extraction_after_RSM/r/call/0dab2bc0-21c3-4e83-87ac-30335c7b9f4e
arms_num:  2
arms_list:  ['Brief Behavioral Treatment for Insomnia (BBTI)', 'Waitlist control']
arm_num:  0
🍩 https://wandb.ai/srws-psg/data_extraction_after_RSM/r/call/a9f636a8-b9d4-4ab3-86af-17d5aaa5a024
{'Year': 2021, 'ind_clu': 'individual', 'ICC_for_cRCT': 0.05, 'Country': 'USA', 'Single_multi': 'single', 'Age_mean': 21, 'Age_sd': 4.85, 'Age_n': 28, 'F_n': 16, 'Hyp_n': 27, 'Insomnia_diagnosis': '2. informal_ISI', 'Primary_insomnia_n': 15, 'Comorbidities': '2. psy_mixed', 'se': 1, 'sd': 1, 'cr': 0, 'cw': 0, 'sr': 1, 'sc': 1, 're': 0, 'pi': 0, 'w': 0, 'dt': 0, 'ns': 0, 'he': 0, 'tg': 1, 'ind': 1, 'gp': 0, 'ff': 1, 'ae': 0, 'n': 28, 'remission': 13, 'def_remission_response': 'BBTI was more efficacious than waitlist control in reducing insomnia symptoms with large effects that participants maintained three months post-treatment at levels correspon

Exception in thread Thread-10 (_process_batches):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/weave/trace_server/async_batch_processor.py", line 61, in _process_batches
    self.processor_fn(current_batch)
  File "/usr/local/lib/python3.10/dist-packages/tenacity/__init__.py", line 332, in wrapped_f
    return self(f, *args, **kw)
  File "/usr/local/lib/python3.10/dist-packages/tenacity/__init__.py", line 469, in __call__
    do = self.iter(retry_state=retry_state)
  File "/usr/local/lib/python3.10/dist-packages/tenacity/__init__.py", line 370, in iter
    result = action(retry_state)
  File "/usr/local/lib/python3.10/dist-packages/tenacity/__init__.py", line 392, in <lambda>
    self._add_action_func(lambda rs: rs.outcome.result())
  File "/usr/lib/python3

🍩 https://wandb.ai/srws-psg/data_extraction_after_RSM/r/call/385b575e-a142-42f5-a45f-0af7cbcb350b
{'Year': 2015, 'ind_clu': 'individual', 'ICC_for_cRCT': 0.05, 'Country': 'Australia', 'Single_multi': 'single', 'Age_mean': 37.1, 'Age_sd': 12.8, 'Age_n': 21, 'F_n': 13, 'Hyp_n': 21, 'Insomnia_diagnosis': '1. formal_DSM', 'Primary_insomnia_n': 0, 'Comorbidities': '2. psy_MDD', 'se': 1, 'sd': 1, 'cr': 1, 'cw': 1, 'sr': 1, 'sc': 1, 're': 1, 'pi': 1, 'w': 0, 'dt': 0, 'ns': 1, 'he': 1, 'tg': 1, 'ind': 1, 'gp': 0, 'ff': 1, 'ae': 0, 'n': 21, 'remission': 13, 'def_remission_response': 'Clinical remission is defined as an ISI score of <8 and depression (BDI-II < 14) were used to establish clinically significant change.', 'scale_used': 'ISI', 'Severity_bl_mean': 18.95, 'Severity_bl_sd': 3.53, 'Severity_bl_n': 21, 'Severity_ch_mean': -11.14, 'Severity_ch_sd': 4.95, 'Severity_ch_n': 21, 'Severity_ep_mean': 7.81, 'Severity_ep_sd': 6.53, 'Severity_ep_n': 21, 'Severity_scale': 'ISI', 'dropout': 3, 'SE_m

#Re-extract prompting

In [ ]:
#Specify developed description obtained by which method is used
shot_name = "chat"
#Specify the path where the results are to be stored
log_dir_name = "extracted_data_log_GPT_v7"

# Iterate over the number of evaluation data and folds
for n_index in range(5,6):
  for fold_index in range(10):
    for eval_index in range(5):
      eval_name=eval_trials[n_index][fold_index][eval_index]
      print("n_index: ", n_index)
      print("fold_index: ", fold_index)
      print("eval_index: ", eval_index)
      extracted_data = {} #For storing extracted data
      output_extracted_data = {} #For storing output data
############################################
      # Define tools for extracting arm information
      tools = [{'type': 'function',
          'function':
              {'name': 'extract_arm',
              'description': 'Extract the number of the arms and each arm name',
              'parameters': {
                  'type': 'object',
                'properties': {
                    "arms_num":
                    {"type": "number",
                    "description": "Answer the number of the arms"
                    },
                    "arm_name_1":
                    {"type": "string",
                    "description": "Answer arm name. If exceed the number of the arms, answer 'nan'."
                    },

                    "arm_name_2":
                    {"type": "string",
                    "description": "Answer arm name. If exceed the number of the arms, answer 'nan'."
                    },

                    "arm_name_3":
                    {"type": "string",
                    "description": "Answer arm name. If exceed the number of the arms, answer 'nan'."
                    },

                    "arm_name_4":
                    {"type": "string",
                    "description": "Answer arm name. If exceed the number of the arms, answer 'nan'."
                    },

                    "arm_name_5":
                    {"type": "string",
                    "description": "Answer arm name. If exceed the number of the arms, answer 'nan'."
                    }
                },
              "required": [
                  "arms_num",
                  "arm_name_1",
                  "arm_name_2",
                  "arm_name_3",
                  "arm_name_4",
                  "arm_name_5"
                  ]
              }
          }
      }
      ]
#######################################################
      # Create messages for extracting arm information
      messages_arm=[]
      system_content = """You are a systematic reviewer.You have to extract arms(i.e., interventions and controls) from the inputted paper of a randomized controlled trial.\n
      Use the extract_arm tool in your response.      """
      user_content = """You have to extract arms(i.e., interventions and controls) from the inputted paper of a randomized controlled trial.\n
      Use the extract_arm tool in your response.
      """
      png_files_sum = 0

      # Create input text from the paper
      root_dir = f"IncludedTrials/{eval_name}" # Root directory containing the trial files
      file_to_find = "structuredData.json" # JSON file to find in directories

      user_content += f"paper_name: {eval_name} \n"
      for dirpath, dirnames, filenames in os.walk(root_dir):# Process each document in the directory
        if (dirpath == root_dir) or (dirpath == root_dir+f"/{eval_name}"):# Only main documents
          if file_to_find in filenames:
            main_paper_txt= extract_sentence_text(dirpath)
            user_content += main_paper_txt

      # Check if main_paper_txt is empty
      if main_paper_txt == "":
        # Generate the log file name, including date and time
        log_file_name = f"{log_dir_name}/{shot_name}/{n_index}_paper/{fold_index}_fold/error_log_{eval_index}_no_paper_text.txt"
        # Define the error message
        error_message = f"An error occurred: no_paper_text"
        print(error_message)
        # Write the error message to the log file
        with open(log_file_name, 'w') as file:
            file.write(error_message)
        # Skip to the next evaluation RCT
        continue;

      messages_arm.append({"role": "system", "content": system_content})
      messages_arm.append({"role": "user", "content": user_content})

############################################
      # Get response from GPT model
      try:
        tools_name = "extract_arm"
        response_arm = GPT_function_calling(tools, messages_arm, tools_name)
      except Exception as e:
        log_file_name = f"{log_dir_name}/{shot_name}/{n_index}_paper/{fold_index}_fold/error_log_{eval_index}_response_arm.txt"
        error_message = f"An error occurred: {str(e)}"
        print(error_message)
        with open(log_file_name, 'w') as file:
            file.write(error_message)
        continue;

############################################
      # Parse the response
      try:
        # Load the JSON response from the GPT output
        output_dict_arm = json.loads(response_arm.choices[0].message.tool_calls[0].function.arguments)#文章で出力されるため、改行で区切る
        # Extract the number of arms from the JSON dictionary
        arms_num = output_dict_arm["arms_num"]

        arms_list = []
        # Loop through each arm and append its name to the arms_list
        for arm_num in range(arms_num):
          arms_list.append(output_dict_arm[f"arm_name_{arm_num+1}"])
        print("arms_num: ", arms_num)
        print("arms_list: ", arms_list)

      except Exception as e:
        log_file_name = f"{log_dir_name}/{shot_name}/{n_index}_paper/{fold_index}_fold/error_log_{eval_index}_output_dict_arm.txt"
        error_message = f"An error occurred: {str(e)}"
        print(error_message)
        with open(log_file_name, 'w') as file:
            file.write(error_message)
        continue;

############################################
      # Process each arm
      for arm_num in range(arms_num):
        print("arm_num: ", arm_num)
############################################
        # Create a dictionary to store the output
        # Define the path to the JSON file
        json_file_path = 'original_description_insomnia.json'

        # Load the JSON data from the file
        with open(json_file_path, 'r') as file:
            data = json.load(file)
        all_fields = [field for field in data]

        # Initialize the output dictionary with field names as keys and "None" as values
        output = {item['name']: "None" for item in all_fields}

############################################
        # Create tools from optimized description
        try:
          # Determine the log path based on n_index (the number of RCTs for training data)
          if n_index == 0:
            log_path = f'optimized_description_log_v3/{shot_name}/{n_index}_paper/{fold_index}_fold/optimized_all_fields_{n_index}*.json'
          else:
            log_path = f'optimized_description_log_v3/{shot_name}/{n_index}_paper/{fold_index}_fold/optimized_all_fields_{n_index-1}*.json'
          file_paths = glob.glob(log_path)

          for file_path in file_paths:
              with open(file_path, 'r') as file:
                  optimized_all_fields = json.load(file)

          name_list = [item['name'] for item in optimized_all_fields]

          #Add options
          for item in optimized_all_fields:
            if item['name'] == 'ind_clu':
              if not('[individual, cluster]' in item['description']):
                item['description'] += ' You have to choose from the options [individual, cluster].'
            if item['name'] == 'Single_multi':
              if not('[single, multi]' in item['description']):
                item['description'] += ' You have to choose from the options [single, multi].'
            if item['name'] == 'Insomnia_diagnosis':
              if not('[1. formal_DSM, 1. formal_ICSD, 1. formal_ICD, 1. formal_Edinger2004, 2. informal_ISI, 2. informal_SCI, 2. informal_PSQI, 2. informal_AIS, 2. informal_other_self_reported_scales, 3. others]' in item['description']):
                item['description'] += ' You have to choose from the options [1. formal_DSM, 1. formal_ICSD, 1. formal_ICD, 1. formal_Edinger2004, 2. informal_ISI, 2. informal_SCI, 2. informal_PSQI, 2. informal_AIS, 2. informal_other_self_reported_scales, 3. others].'
            if item['name'] == 'Comorbidities':
              if not('[0. primary_insomnia, 1. primary_secondary_mixed, 2. psy_MDD, 2. psy_anxiety, 2. psy_PTSD, 2. psy_schizophrenia_or_psychosis, 2. psy_bipolar, 2. psy_substance_dependence, 2. psy_others, 2. psy_mixed, 3. phy_dialysis, 3. phy_heart_failure, 3. phy_diabetes, 3. phy_fibromyalgia, 3. phy_cancer, 3. phy_steroid, 3. phy_pregnancy_perinatal, 3. phy_menopause, 3. phy_others, 3. phy_mixed, 4. psy_phy_mixed]' in item['description']):
                item['description'] += ' You have to choose from the options [0. primary_insomnia, 1. primary_secondary_mixed, 2. psy_MDD, 2. psy_anxiety, 2. psy_PTSD, 2. psy_schizophrenia_or_psychosis, 2. psy_bipolar, 2. psy_substance_dependence, 2. psy_others, 2. psy_mixed, 3. phy_dialysis, 3. phy_heart_failure, 3. phy_diabetes, 3. phy_fibromyalgia, 3. phy_cancer, 3. phy_steroid, 3. phy_pregnancy_perinatal, 3. phy_menopause, 3. phy_others, 3. phy_mixed, 4. psy_phy_mixed].'
            if item['name'] == 'scale_used':
              if not('[ISI, SCI, FOSQ, ESS, PSQI, AIS, other_self_reported_scales, WASO_SOL, WASO, SOL]' in item['description']):
                item['description'] += ' You have to choose from the options [ISI, SCI, FOSQ, ESS, PSQI, AIS, other_self_reported_scales, WASO_SOL, WASO, SOL].'
            if item['name'] == 'Severity_scale':
              if not('[ISI, SCI, FOSQ, ESS, PSQI, AIS, other_self_reported_scales, WASO_SOL, WASO, SOL]' in item['description']):
                item['description'] += ' You have to choose from the options [ISI, SCI, FOSQ, ESS, PSQI, AIS, other_self_reported_scales, WASO_SOL, WASO, SOL].'
            if item['name'] == 'r_long_scale_used':
              if not('[ISI, SCI, FOSQ, ESS, PSQI, AIS, other_self_reported_scales, WASO_SOL, WASO, SOL]' in item['description']):
                item['description'] += ' You have to choose from the options [ISI, SCI, FOSQ, ESS, PSQI, AIS, other_self_reported_scales, WASO_SOL, WASO, SOL].'

        except Exception as e:
          extracted_data[arms_list[arm_num]] = copy.deepcopy(output)
          log_file_name = f"{log_dir_name}/{shot_name}/{n_index}_paper/{fold_index}_fold/error_log_{eval_index}_data_extraction_armnum_{arm_num}.txt"
          error_message = f"An error occurred: {str(e)}"
          print(error_message)
          with open(log_file_name, 'w') as file:
              file.write(error_message)
          continue;

############################################
        #Create tools and messages for data extraction
        description_tools = make_tools_DE(optimized_all_fields)

        messages_DE = []
        system_content_DE = """You are a systematic reviewer and extract data from RCT.
        """
        query_DE = f"""You have to extract features in the {arms_list[arm_num]} arm from the inputted paper of a randomized controlled trial.
        Use the extract_study_features tool in your response.\n
        """
        query_DE += attention_query
        query_DE += f"trial_name: {eval_name} \n \n "
        query_DE += "The extract_study_features:"
        query_DE += json.dumps(description_tools)

        last_query = attention_query

        messages_GPT = []
        content_prompt_GPT = [] # List to hold content prompts for GPT

        png_files_sum = 0 # Sum of PNG files processed

        # Create input text from the paper
        root_dir = f"IncludedTrials/{eval_name}" # Root directory containing the trial files
        file_to_find = "structuredData.json" # JSON file to find in directories

        content_prompt_GPT.append({"type": "text", "text": query_DE})

        # Extract sentences from each document in the directory
        for dirpath, dirnames, filenames in os.walk(root_dir):
          if eval_name in dirnames:# Process the main paper first
            dirnames.remove(eval_name)
            dirnames.insert(0, eval_name)
          if file_to_find in filenames:
            paper_query = dirpath.split("/")[-1]
            paper_query += extract_sentence(dirpath)
            content_prompt_GPT.append({"type": "text", "text":paper_query})

            # Filter and add PNG files to the content prompt
            if os.path.exists(f"{dirpath}/figures"):
              png_files = [file for file in os.listdir(f"{dirpath}/figures") if file.endswith('.png')]

              for i in range(len(png_files)):
                if png_files_sum >= 10:# Limit the number of images to 10
                  break;
                content_prompt_GPT.append({"type": "image_url", "image_url": {"url": local_image_to_data_url(f"{dirpath}/figures/"+png_files[i])}})
                png_files_sum += 1

        # Process txt and docx files in the root directory
        txt_docx_query = "" # String to hold txt and docx file contents
        txt_docx_flag = False # Flag to check if txt or docx files are found

        for file_name in os.listdir(root_dir):
            if file_name.endswith('.txt'):
                txt_docx_flag = True
                txt_docx_query += "txt_name: "
                txt_docx_query += "txt_name: " + file_name + "\n"
                file_path = os.path.join(root_dir, file_name)
                with open(file_path, 'r', encoding='utf-8') as file:
                    txt_docx_query += file.read()
            elif file_name.endswith('.docx'):
                txt_docx_flag = True
                txt_docx_query += "docx_name: "
                txt_docx_query += "docx_name: " + file_name + "\n"
                file_path = os.path.join(root_dir, file_name)
                doc = Document(file_path)
                full_text = []
                for para in doc.paragraphs:
                    full_text.append(para.text)

                for table in doc.tables:
                    for row in table.rows:
                        for cell in row.cells:
                            full_text.append(cell.text)
                txt_docx_query += '\n'.join(full_text)

        if txt_docx_flag:
          content_prompt_GPT.append({"type": "text", "text":txt_docx_query})


        content_prompt_GPT.append({"type": "text", "text": last_query})

        messages_GPT.append({"role": "system", "content": system_content_DE})
        messages_GPT.append({"role": "user", "content": content_prompt_GPT})

############################################
        # Get response from GPT model
        tools_name = "extract_study_features"
        response_GPT = GPT_chat(description_tools, messages_GPT, tools_name)

        # Store the results
        output_dict = json.loads(response_GPT.choices[0].message.tool_calls[0].function.arguments)

        DE_name_list = []
        Non_DE_name_list = []
        print(output_dict)

        # Update the output dictionary with values from output_dict
        for key in output_dict.keys():
          if str(output_dict[key]) != "-1":
            DE_name_list.append(key)
          else:
            Non_DE_name_list.append(key)
            output[key] = -1
        print("DE_name_list: ", DE_name_list)
        print("Non_DE_name_list: ", Non_DE_name_list)

    ############################################
    #Data extraction again for each of the four variables for which data has been extracted (Re-extract)

        # Initialize messages and a list to track differences for DE extraction
        messages_DE = []
        different_DE_dict = []

        # Define the system content for DE extraction
        system_content_DE = """You are a systematic reviewer.
        Use the extract_study_features tool in your response.
        """
        query_DE = f"""You have to extract features in the {arms_list[arm_num]} arm from the inputted paper of a randomized controlled trial.
        Use the extract_study_features tool in your response.\n
        """
        query_DE += attention_query
        query_DE += f"trial_name: {eval_name} \n \n "

        # Create and modify the content prompt for DE extraction
        content_prompt_DE = copy.deepcopy(content_prompt_GPT)
        content_prompt_DE.pop(0)
        content_prompt_DE.insert(0, {"type": "text", "text": query_DE})

        # Append initial system, user and assistant messages to messages_DE
        messages_DE.append({"role": "system", "content": system_content_DE})
        messages_DE.append({"role": "user", "content": content_prompt_DE})
        messages_DE.append({"role": "assistant", "content": "Sure. What variable do you want to extract data from?"})

        # Loop through DE_name_list in chunks of 4 and perform data extraction
        for split in range(((len(DE_name_list))//4+1)):
          start = split*4
          end = min(len(DE_name_list), (split+1)*4)

          # Create description tools for the current chunk of DE_name_list
          description_tools = make_tools_DE_recheck(optimized_all_fields,DE_name_list, start, end)
          tools_name = "extract_study_features"

          messages_DE.append({"role": "user", "content": f"Please extract the data for {DE_name_list}"})

          # Call GPT function for DE extraction and append the response to messages_DE
          response_DE = GPT_chat(description_tools, messages_DE, tools_name)
          messages_DE.append({"role":"assistant", "content": response_DE.choices[0].message.tool_calls[0].function.arguments})

          # Parse the JSON response from DE extraction
          output_DE_dict = json.loads(response_DE.choices[0].message.tool_calls[0].function.arguments)

          # Store the obtained results and track differences
          for key in output_DE_dict.keys():
              if key in output.keys():
                  output[key] = output_DE_dict[key]
              if output_DE_dict[key] != output_dict[key]:
                different_DE_dict.append(key)

        # Store the updated output in extracted_data with the current arm name
        extracted_data[arms_list[arm_num]] = copy.deepcopy(output)
        print(different_DE_dict)
      ############################################
      # Write the extraction results of the current evaluation RCT to a JSON file
      file_name = f"{log_dir_name}/{shot_name}/{n_index}_paper/{fold_index}_fold/extracted_data_{eval_index}.json"
      output_extracted_data[eval_name] = copy.deepcopy(extracted_data)
      print("log has been created: ", extracted_data)

      # Write the output_extracted_data to the specified JSON file
      with open(file_name, 'w') as file:
          json.dump(output_extracted_data, file, indent=4)


n_index:  5
fold_index:  0
eval_index:  1
🍩 https://wandb.ai/srws-psg/data_extraction_after_RSM/r/call/a96dd9d2-cbac-48c7-ad82-f3eaa6e8f605
arms_num:  2
arms_list:  ['CBT plus standard care', 'standard care alone']
arm_num:  0
🍩 https://wandb.ai/srws-psg/data_extraction_after_RSM/r/call/5d4c4a3f-9ed4-4792-9691-0192f015aa29
{'Year': 2015, 'ind_clu': 'individual', 'ICC_for_cRCT': 0.05, 'Country': 'UK', 'Single_multi': 'multi', 'Age_mean': 39.6, 'Age_sd': 11.6, 'Age_n': 24, 'F_n': 8, 'Hyp_n': 6, 'Insomnia_diagnosis': '2. informal_ISI', 'Primary_insomnia_n': -1, 'Comorbidities': '2. psy_schizophrenia_or_psychosis', 'se': 1, 'sd': 1, 'cr': 1, 'cw': 0, 'sr': 1, 'sc': 1, 're': 1, 'pi': 0, 'w': 0, 'dt': 1, 'ns': 0, 'he': 0, 'tg': 1, 'ind': 1, 'gp': 0, 'ff': 1, 'ae': 0, 'n': 24, 'remission': 9, 'def_remission_response': 'By week 12, nine (41%) of 22 patients receiving CBT and one (4%) of 25 patients receiving standard care alone no longer had insomnia, with ISI scores lower than the cutoff for 

#Batch data extraction


In [ ]:
weave.init("data_extraction_after_RSM")

In [ ]:
@weave.op() # 🐝
def GPT_chat(tools, messages, tools_name):
  response = client.chat.completions.create(
        model=deployment_ids[0],
        messages=messages,
        tools=tools,
        tool_choice =  {"type": "function", "function": {"name": tools_name}},
        temperature=0
    )
  return response

In [ ]:
#Specify developed description obtained by which method is used
shot_name = "chat"
#Specify the path where the results are to be stored
log_dir_name = "extracted_data_log_GPT_v4"

# Iterate over the number of evaluation data and folds
for n_index in range(5,6):
  for fold_index in range(1):
    for eval_index in range(5):
      eval_name=eval_trials[n_index][fold_index][eval_index]
      print("n_index: ", n_index)
      print("fold_index: ", fold_index)
      print("eval_index: ", eval_index)

      extracted_data = {} #For storing extracted data
      output_extracted_data = {} #For storing output data
############################################
      # Define tools for extracting arm information
      tools = [{'type': 'function',
          'function':
              {'name': 'extract_arm',
              'description': 'Extract the number of the arms and each arm name',
              'parameters': {
                  'type': 'object',
                'properties': {
                    "arms_num":
                    {"type": "number",
                    "description": "Answer the number of the arms"
                    },
                    "arm_name_1":
                    {"type": "string",
                    "description": "Answer arm name. If exceed the number of the arms, answer 'nan'."
                    },

                    "arm_name_2":
                    {"type": "string",
                    "description": "Answer arm name. If exceed the number of the arms, answer 'nan'."
                    },

                    "arm_name_3":
                    {"type": "string",
                    "description": "Answer arm name. If exceed the number of the arms, answer 'nan'."
                    },

                    "arm_name_4":
                    {"type": "string",
                    "description": "Answer arm name. If exceed the number of the arms, answer 'nan'."
                    },

                    "arm_name_5":
                    {"type": "string",
                    "description": "Answer arm name. If exceed the number of the arms, answer 'nan'."
                    }
                },
              "required": [
                  "arms_num",
                  "arm_name_1",
                  "arm_name_2",
                  "arm_name_3",
                  "arm_name_4",
                  "arm_name_5"
                  ]
              }
          }
      }
      ]
#######################################################
      # Create messages for extracting arm information
      messages_arm=[]
      system_content = """You are a systematic reviewer.You have to extract arms(i.e., interventions and controls) from the inputted paper of a randomized controlled trial.\n
      Use the extract_arm tool in your response.\n\n"""
      user_content = """You have to extract arms(i.e., interventions and controls) from the inputted paper of a randomized controlled trial.\n
      Use the extract_arm tool in your response.\n\n"""
      png_files_sum = 0

      # Create input text from the paper
      root_dir = f"IncludedTrials/{eval_name}"# Root directory containing the trial files
      file_to_find = "structuredData.json" # JSON file to find in directories

      user_content += f"paper_name: {eval_name} \n"
      for dirpath, dirnames, filenames in os.walk(root_dir):# Process each document in the directory
        if (dirpath == root_dir) or (dirpath == root_dir+f"/{eval_name}"):# Only main documents
          if file_to_find in filenames:
            main_paper_txt= extract_sentence_text(dirpath)
            user_content += main_paper_txt

      # Check if main_paper_txt is empty
      if main_paper_txt == "":
        # Generate the log file name, including date and time
        log_file_name = f"{log_dir_name}/{shot_name}/{n_index}_paper/{fold_index}_fold/error_log_{eval_index}_no_paper_text.txt"
        # Define the error message
        error_message = f"An error occurred: no_paper_text"
        print(error_message)
        # Write the error message to the log file
        with open(log_file_name, 'w') as file:
            file.write(error_message)
        # Skip to the next evaluation RCT
        continue;

      messages_arm.append({"role": "system", "content": system_content})
      messages_arm.append({"role": "user", "content": user_content})

############################################
      # Get response from GPT model
      try:
        tools_name = "extract_arm"
        response_arm = GPT_function_calling(tools, messages_arm, tools_name)
      except Exception as e:
        log_file_name = f"{log_dir_name}/{shot_name}/{n_index}_paper/{fold_index}_fold/error_log_{eval_index}_response_arm.txt"
        error_message = f"An error occurred: {str(e)}"
        print(error_message)
        with open(log_file_name, 'w') as file:
            file.write(error_message)
        continue;

############################################
      # Parse the response
      try:
        # Load the JSON response from the GPT output
        output_dict_arm = json.loads(response_arm.choices[0].message.tool_calls[0].function.arguments)
        # Extract the number of arms from the JSON dictionary
        arms_num = output_dict_arm["arms_num"]
        arms_list = []

        # Loop through each arm and append its name to the arms_list
        for arm_num in range(arms_num):
          arms_list.append(output_dict_arm[f"arm_name_{arm_num+1}"])
        print("arms_num: ", arms_num)
        print("arms_list: ", arms_list)

      except Exception as e:
        log_file_name = f"{log_dir_name}/{shot_name}/{n_index}_paper/{fold_index}_fold/error_log_{eval_index}_output_dict_arm.txt"
        error_message = f"An error occurred: {str(e)}"
        print(error_message)
        with open(log_file_name, 'w') as file:
            file.write(error_message)
        continue;

############################################
      # Process each arm
      for arm_num in range(arms_num):
        print("arm_num: ", arm_num)
############################################
        # Create a dictionary to store the output
        # Define the path to the JSON file
        json_file_path = 'original_description_insomnia.json'

        # Load the JSON data from the file
        with open(json_file_path, 'r') as file:
            data = json.load(file)

        all_fields = [field for field in data]
        # Initialize the output dictionary with field names as keys and "None" as values
        output = {item['name']: "None" for item in all_fields}

############################################
        # Create tools from optimized description
        try:
          # Determine the log path based on n_index (the number of RCTs for training data)
          if n_index == 0:
            log_path = f'optimized_description_log_v3/{shot_name}/{n_index}_paper/{fold_index}_fold/optimized_all_fields_{n_index}*.json'
          else:
            log_path = f'optimized_description_log_v3/{shot_name}/{n_index}_paper/{fold_index}_fold/optimized_all_fields_{n_index-1}*.json'
          file_paths = glob.glob(log_path)

          for file_path in file_paths:
              with open(file_path, 'r') as file:
                  optimized_all_fields = json.load(file)

          #Add options
          for item in optimized_all_fields:
            if item['name'] == 'ind_clu':
              if not('[individual, cluster]' in item['description']):
                item['description'] += ' You have to choose from the options [individual, cluster].'
            if item['name'] == 'Single_multi':
              if not('[single, multi]' in item['description']):
                item['description'] += ' You have to choose from the options [single, multi].'
            if item['name'] == 'Insomnia_diagnosis':
              if not('[1. formal_DSM, 1. formal_ICSD, 1. formal_ICD, 1. formal_Edinger2004, 2. informal_ISI, 2. informal_SCI, 2. informal_PSQI, 2. informal_AIS, 2. informal_other_self_reported_scales, 3. others]' in item['description']):
                item['description'] += ' You have to choose from the options [1. formal_DSM, 1. formal_ICSD, 1. formal_ICD, 1. formal_Edinger2004, 2. informal_ISI, 2. informal_SCI, 2. informal_PSQI, 2. informal_AIS, 2. informal_other_self_reported_scales, 3. others].'
            if item['name'] == 'Comorbidities':
              if not('[0. primary_insomnia, 1. primary_secondary_mixed, 2. psy_MDD, 2. psy_anxiety, 2. psy_PTSD, 2. psy_schizophrenia_or_psychosis, 2. psy_bipolar, 2. psy_substance_dependence, 2. psy_others, 2. psy_mixed, 3. phy_dialysis, 3. phy_heart_failure, 3. phy_diabetes, 3. phy_fibromyalgia, 3. phy_cancer, 3. phy_steroid, 3. phy_pregnancy_perinatal, 3. phy_menopause, 3. phy_others, 3. phy_mixed, 4. psy_phy_mixed]' in item['description']):
                item['description'] += ' You have to choose from the options [0. primary_insomnia, 1. primary_secondary_mixed, 2. psy_MDD, 2. psy_anxiety, 2. psy_PTSD, 2. psy_schizophrenia_or_psychosis, 2. psy_bipolar, 2. psy_substance_dependence, 2. psy_others, 2. psy_mixed, 3. phy_dialysis, 3. phy_heart_failure, 3. phy_diabetes, 3. phy_fibromyalgia, 3. phy_cancer, 3. phy_steroid, 3. phy_pregnancy_perinatal, 3. phy_menopause, 3. phy_others, 3. phy_mixed, 4. psy_phy_mixed].'
            if item['name'] == 'scale_used':
              if not('[ISI, SCI, FOSQ, ESS, PSQI, AIS, other_self_reported_scales, WASO_SOL, WASO, SOL]' in item['description']):
                item['description'] += ' You have to choose from the options [ISI, SCI, FOSQ, ESS, PSQI, AIS, other_self_reported_scales, WASO_SOL, WASO, SOL].'
            if item['name'] == 'Severity_scale':
              if not('[ISI, SCI, FOSQ, ESS, PSQI, AIS, other_self_reported_scales, WASO_SOL, WASO, SOL]' in item['description']):
                item['description'] += ' You have to choose from the options [ISI, SCI, FOSQ, ESS, PSQI, AIS, other_self_reported_scales, WASO_SOL, WASO, SOL].'
            if item['name'] == 'r_long_scale_used':
              if not('[ISI, SCI, FOSQ, ESS, PSQI, AIS, other_self_reported_scales, WASO_SOL, WASO, SOL]' in item['description']):
                item['description'] += ' You have to choose from the options [ISI, SCI, FOSQ, ESS, PSQI, AIS, other_self_reported_scales, WASO_SOL, WASO, SOL].'

        except Exception as e:
          extracted_data[arms_list[arm_num]] = copy.deepcopy(output)
          log_file_name = f"{log_dir_name}/{shot_name}/{n_index}_paper/{fold_index}_fold/error_log_{eval_index}_data_extraction_armnum_{arm_num}.txt"
          error_message = f"An error occurred: {str(e)}"
          print(error_message)
          with open(log_file_name, 'w') as file:
              file.write(error_message)
          continue;


############################################
        #Process every four variables
        for split in range(17):
          #Convert to appropriate structure for incorporation into tools
          properties_dict = {item['name']: {k: v for k, v in item.items() if k != 'name'} for item in optimized_all_fields}
          name_list = [item['name'] for item in optimized_all_fields]

          # Define the initial structure of tools for data extraction
          description_tools = [{'type': 'function',
              'function':
                  {'name': 'extract_study_features',
                  'description': 'Extracts key features from a user-inputted research article for systematic review purposes.',
                  'parameters': {
                      'type': 'object',
                    'properties': {}
                  }
              }
          }
          ]
          # Split the name list into four parts
          start = split*4
          end = min(67, (split+1)*4)
          splitted_name_list = copy.deepcopy(name_list[start:end])
          description_tools[0]['function']['parameters']['required'] = splitted_name_list
          description_tools[0]['function']['parameters']['properties'] = {key: properties_dict[key] for key in splitted_name_list if key in properties_dict}

  ############################################
          # Create messages for data extraction
          system_content_DE = """You are a systematic reviewer.
          Use the extract_study_features tool in your response.
          """

          query_GPT = f"""You have to extract features in the {arms_list[arm_num]} arm from the inputted paper of a randomized controlled trial.
            Use the extract_study_features tool in your response.\n
              """
          query_GPT += attention_query
          query_GPT += f"trial_name: {eval_name} \n \n "

          query_function_calling = copy.deepcopy(query_GPT)

          query_GPT += "The extract_study_features:"
          query_GPT += json.dumps(description_tools)

          last_query = attention_query

          messages_GPT = []
          content_prompt_GPT = [] # List to hold content prompts for GPT

          png_files_sum = 0 # Sum of PNG files processed

          # Create input text from the paper
          root_dir = f"IncludedTrials/{eval_name}" # Root directory containing the trial files
          file_to_find = "structuredData.json" # JSON file to find in directories

          content_prompt_GPT.append({"type": "text", "text": query_GPT})

          # Extract sentences from each document in the directory
          for dirpath, dirnames, filenames in os.walk(root_dir):
            if eval_name in dirnames:# Process the main paper first
              dirnames.remove(eval_name)
              dirnames.insert(0, eval_name)
            if file_to_find in filenames:
              paper_query = dirpath.split("/")[-1]
              paper_query += extract_sentence(dirpath)
              content_prompt_GPT.append({"type": "text", "text":paper_query})

              # Filter and add PNG files to the content prompt
              if os.path.exists(f"{dirpath}/figures"):
                png_files = [file for file in os.listdir(f"{dirpath}/figures") if file.endswith('.png')]

                for i in range(len(png_files)):
                  if png_files_sum >= 10:# Limit the number of images to 10
                    break;
                  content_prompt_GPT.append({"type": "image_url", "image_url": {"url": local_image_to_data_url(f"{dirpath}/figures/"+png_files[i])}})
                  png_files_sum += 1

          # Process txt and docx files in the root directory
          txt_docx_query = "" # String to hold txt and docx file contents
          txt_docx_flag = False # Flag to check if txt or docx files are found

          for file_name in os.listdir(root_dir):
              if file_name.endswith('.txt'):
                  txt_docx_flag = True
                  txt_docx_query += "txt_name: "
                  txt_docx_query += "txt_name: " + file_name + "\n"
                  file_path = os.path.join(root_dir, file_name)
                  with open(file_path, 'r', encoding='utf-8') as file:
                      txt_docx_query += file.read()
              elif file_name.endswith('.docx'):
                  txt_docx_flag = True
                  txt_docx_query += "docx_name: "
                  txt_docx_query += "docx_name: " + file_name + "\n"
                  file_path = os.path.join(root_dir, file_name)
                  doc = Document(file_path)
                  full_text = []
                  for para in doc.paragraphs:
                      full_text.append(para.text)

                  for table in doc.tables:
                      for row in table.rows:
                          for cell in row.cells:
                              full_text.append(cell.text)
                  txt_docx_query += '\n'.join(full_text)

          if txt_docx_flag:
            content_prompt_GPT.append({"type": "text", "text":txt_docx_query})


          content_prompt_GPT.append({"type": "text", "text": last_query})

          messages_GPT.append({"role": "system", "content": system_content_DE})
          messages_GPT.append({"role": "user", "content": content_prompt_GPT})

      ############################################
          # Get response from GPT model
          tools_name = "extract_study_features"
          response_GPT = GPT_chat(description_tools, messages_GPT, tools_name)
          output_dict = json.loads(response_GPT.choices[0].message.tool_calls[0].function.arguments)
          # Store the results
          for key in output_dict.keys():
              if key in output.keys():
                  if str(output_dict[key]) != "-1":
                      output[key] = output_dict[key]
    ############################################
    # Extract individually any data that was not extracted
          recheck_name_list = [key for key in output_dict.keys() if str(output_dict[key]) == "-1"]
          print("recheck_name_list: ", recheck_name_list)

          for recheck_name in recheck_name_list:
            # Re-define the initial structure of tools for data extraction
              properties_dict = {item['name']: {k: v for k, v in item.items() if k != 'name'} for item in optimized_all_fields}
              name_list = [recheck_name]
              description_tools = [{'type': 'function',
                                    'function': {
                                        'name': 'extract_study_features',
                                        'description': 'Extracts key features from a user-inputted research article for systematic review purposes.',
                                        'parameters': {
                                            'type': 'object',
                                            'properties': {}
                                        }
                                    }
                                    }
                                  ]
              splitted_name_list = copy.deepcopy(name_list)
              description_tools[0]['function']['parameters']['required'] = splitted_name_list
              description_tools[0]['function']['parameters']['properties'] = {key: properties_dict[key] for key in splitted_name_list if key in properties_dict}

              # Create the messages for extract
              messages_recheck = []
              system_content_check = """You are a systematic reviewer and extract data from RCT."""
              query_recheck = f"""You have to extract features in the {arms_list[arm_num]} arm from the inputted paper of a randomized controlled trial.
              Use the extract_study_features tool in your response.\n
              """
              query_recheck += attention_query
              query_recheck += f"trial_name: {eval_name} \n \n "
              query_recheck += "The extract_study_features:"
              query_recheck += json.dumps(description_tools)

              content_prompt_recheck = copy.deepcopy(content_prompt_GPT)
              content_prompt_recheck.pop(0)
              content_prompt_recheck.insert(0, {"type": "text", "text": query_recheck})

              messages_recheck.append({"role": "system", "content": system_content_check})
              messages_recheck.append({"role": "user", "content": content_prompt_recheck})

              tools_name = "extract_study_features"

              # Call GPT function for extracting
              response_recheck = GPT_chat(description_tools, messages_recheck, tools_name)
              # Parse the JSON response
              output_recheck_dict = json.loads(response_recheck.choices[0].message.tool_calls[0].function.arguments)

              # Store the obtained results
              for key in output_recheck_dict.keys():
                  if key in output.keys():
                      output[key] = output_recheck_dict[key]

        # Store the updated output in extracted_data with the current arm name
        extracted_data[arms_list[arm_num]] = copy.deepcopy(output)

      ############################################
      # Write the extraction results of the current evaluation paper to a JSON file
      file_name = f"{log_dir_name}/{shot_name}/{n_index}_paper/{fold_index}_fold/extracted_data_{eval_index}.json"
      output_extracted_data[eval_name] = copy.deepcopy(extracted_data)
      print("log has been created: ", extracted_data)

      # Write the output_extracted_data to the specified JSON file
      with open(file_name, 'w') as file:
          json.dump(output_extracted_data, file, indent=4)


n_index:  5
fold_index:  0
eval_index:  0
🍩 https://wandb.ai/srws-psg/data_extraction_after_RSM/r/call/9b124da3-d865-4daf-883d-623e8e3b88ee
arms_num:  2
arms_list:  ['CBT-I', 'MEC']
arm_num:  0
🍩 https://wandb.ai/srws-psg/data_extraction_after_RSM/r/call/2de5322a-cafe-4e17-b11e-981ea213966c
recheck_name_list:  []
🍩 https://wandb.ai/srws-psg/data_extraction_after_RSM/r/call/58b0a846-87f8-45d7-8c1b-04ec941d7005
recheck_name_list:  []
🍩 https://wandb.ai/srws-psg/data_extraction_after_RSM/r/call/2d75834d-1925-46b3-b6c8-e7f2a4661eea
recheck_name_list:  ['Primary_insomnia_n']
🍩 https://wandb.ai/srws-psg/data_extraction_after_RSM/r/call/6b8dd610-b9ec-4cd7-9c72-b5c44bc25da0
🍩 https://wandb.ai/srws-psg/data_extraction_after_RSM/r/call/1ae40cac-6369-402e-a92f-a02c3049bf8d
recheck_name_list:  []
🍩 https://wandb.ai/srws-psg/data_extraction_after_RSM/r/call/3ff19761-a6eb-4968-8bd8-29da4dcb5e6c
recheck_name_list:  []
🍩 https://wandb.ai/srws-psg/data_extraction_after_RSM/r/call/5414c931-5d31-4a21-933